In [1]:
import numpy as np
import pandas as pd
from fractions import Fraction
import xlsxwriter
import decimal
from decimal import *
import re
from modsigfig import round #does this need to be done 
pd.set_option('display.max_rows', 500)
getcontext().prec = 20
#allowed: p to s, d, p. s to p. d to s, d.
# n anything below energy
# j = 1/2 to 1/2, 3/2. j = 3/2 to 3/2, 1/2, 5/2. j = 5/2 to 5/2, 3/2, 7/2. 

In [2]:
%run -i LoadFunctions.py

In [3]:
%run -i Format_save_copy.py

In [4]:
%run -i LoadInElement.py

Element name in files:Fr


C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\LoadInElement.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mats['old_unc'] = database['unc']
C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\LoadInElement.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_state['old_unc'] = database['unc']
C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\LoadInElement.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

Replaced Values, experimental index, all_state index
0 56
1 57


d:\python3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
d:\python3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\LoadInElement.py:638: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [5]:
#tasks: 
#1 add * to nist modified data
#2 mess with variable format so that rouding for save_copy does not occur twice, once to get it in 4 digits and once more for ()
#2.5 change it so E is +04 instead, if the numbers is less than 10
#3 compare numbers with Safronova, including lifetime
#make a rough sketch of the website "other data" page

In [6]:
#problem: branching ratio is Tr1/Sum(Trs), which is TR1 * lifetime. But lifetime changes

In [7]:
# all_state.drop(['Initial_form', 'Decay_form'], axis = 1, inplace = True)

In [8]:

# precise_wave = []
# precise_Eerr = []
# for i in range(len(all_state)): #saves new columns of all_state to be used in calculation, that aren't rounded yet
#     precise_wave.append((1/(all_state.Ei[i] - all_state.Ef[i]))*10**7)
#     precise_Eerr.append(Decimal(all_state.Eerr[i]))
# all_state['precise_wave'] = precise_wave
# all_state['precise_Eerr'] = precise_Eerr

In [9]:
duplicated_error = all_state[all_state.duplicated(['Terr'])]
if len(duplicated_error) == 0:
    print('No duplicated Errors, can run error calc2')
else:
    print('There are duplicated Transition rate Errors')
    raise SystemExit(0)

No duplicated Errors, can run error calc2


In [10]:
og_lifetimes = Lifetimes.copy()

In [11]:
save_copy = all_state.copy() #used to display error formatted numbers in TR table on main page

In [12]:
#BREAK

In [13]:
#At this point:
#Wavelength and Error are in nm
#Transition Rate and Error are in s-1

In [14]:
#Removes 'f' States and above
for i in range(len(save_copy)-1, 0, -1):
    if save_copy.Initial[i][1] >= 3:
        save_copy.drop(i, axis = 0, inplace = True)

In [15]:
#Removes 'f' States and above
for i in range(len(Lifetimes)-1,0,-1): #counting backwards
    if Lifetimes[i][0][1] >= 3:
        del(Lifetimes[i])


In [17]:
#this runs it on all_state and doesn't save to save_copy

pattern = re.compile(r"\((\d+)\)")
holder = []
diff_holder = []
#for p in range(len(save_copy)):
for p in range(279,280):
    ini = all_state.Initial[p]
    end = all_state.Decay[p]
    test_val = all_state.precise_wave[p]
    test_err = all_state.precise_Eerr[p]
    if float(test_err) == 0:
        value_wl = '%.4f' % test_val
    else:
        if float(all_state.wavelength[p]) and float(all_state.Eerr[p]) > 9.5: #need to avoid sci. notat
            #len_figs = the difference between the digit placement of the number and its error. I.e. ####.## - ##.## will be 2
            len_figs = len(str(test_val).split('.')[0]) - len(str(test_err).split('.')[0])

            if float(test_err) < 10: #9.5-10 will round up to 1 and lose a zero digit
                num_w = str(round(str(test_val), sigfigs = len_figs)).split('.')[0] #dont need to add one if it isnt double digits

            else:
                num_w = str(round(str(test_val), len_figs + 1)).split('.')[0]

            s = to_one_dig(test_val, test_err)
            num_e = s[s.find("(")+1:s.find(")")] #number in parentheses
            num_zeroes = len(num_w) - len(num_w.rstrip('0'))
            num_e = '(' + num_e + '0'*num_zeroes + ')'
            value_wl = num_w+num_e
            print(s, num_w, num_e, num_zeroes)

            #value_wl = s.split('(')[0] + num_e
            #print(p, test_val, test_err, value_wl, num_zeroes, num_w)

        else: #small enough to not use sci.not
            one_dig = to_one_dig(test_val, test_err)
            two_dig = round(test_val, test_err, format = 'Drake') 

            if test_err == 0: #if no error use 4 digit wavelength 
                value_wl = all_state.wavelength[p]

            #if two dig has error something like (97), one_dig has error (1). It rounds to 10 but then cuts the 0, which appears wrong
            #needs to be pushed up one digit spot. I.e. (2.45 (97) needs to be 2.5(1))
            elif one_dig[one_dig.find("(")+1:one_dig.find(")")][0] < two_dig[two_dig.find("(")+1:two_dig.find(")")][0]: 
                left_sigfs = len(one_dig.split('(')[0][0:-1]) - 1 #rounding to one less digit instead of slicing for rounding
                #Rounds original number to correct number of sig figs
                value_wl = str(round(test_val, sigfigs = left_sigfs)) + '(' + one_dig[one_dig.find("(")+1:one_dig.find(")")][0] + ')'  
            #otherwise, standard case
            else: 
                #print(test_val, test_err)
                
                #use the original error if the error is less than the .0001 cutoff, since otherwise error will not show up
                if test_err < .0001:
                    value_wl = to_one_dig_small(test_val, float(all_state.Eerr[p]))
                else:
                    value_wl = to_one_dig_small(test_val, test_err)
                #value_wl = round(test_val, test_err, format = 'Drake', err_dig = 1)

        
        

    value_tr = round(str(all_state['transition_rate s-1'][p]), str(all_state.Terr[p]), format = "Drake")
    if 'E' in value_tr:
        pass
    else:
        numt = "{:E}".format(Decimal(value_tr.split('(')[0])).split('E')
        if float(numt[1]) < 10: #need to add '0' in front of value if exponent power is less than 10
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + '+' + '0' + str(int(numt[1]))
            
        else:
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + numt[1]
    
    value_br = round(str(all_state['branching ratio'][p]), str(all_state.Berr[p]), format = "Drake")
    holder.append((value_wl, value_tr, value_br))
print(value_wl, test_val, test_err, all_state['Eerr'][p])
holder

28300.0000(8) 28300 (800) 2
28300(800) 28295.394867894460441 80.064378192904172237831517122685909271240234375 80.0644


[('28300(800)', '6.26(37)E+04', '0.0229(14)')]

In [22]:
bf_format = save_copy.copy()

In [23]:
#one to use if you haven't already modified save_copy
pattern = re.compile(r"\((\d+)\)")
holder = []
for p in range(len(save_copy)):
#for p in range(0, 3):
    ini = save_copy.Initial[p]
    end = save_copy.Decay[p]
    test_val = save_copy.precise_wave[p]
    test_err = save_copy.precise_Eerr[p]
    if float(test_err) == 0:
        value_wl = '%.4f' % test_val
    else:
        if float(save_copy.wavelength[p]) and float(save_copy.Eerr[p]) > 9.5: #need to avoid sci. notat
            #len_figs = the difference between the digit placement of the number and its error. I.e. ####.## - ##.## will be 2
            len_figs = len(str(test_val).split('.')[0]) - len(str(test_err).split('.')[0])

            if float(test_err) < 10: #9.5-10 will round up to 1 and lose a zero digit
                num_w = str(round(str(test_val), sigfigs = len_figs)).split('.')[0] #dont need to add one if it isnt double digits

            else:
                num_w = str(round(str(test_val), len_figs + 1)).split('.')[0]
            s = to_one_dig(test_val, test_err)
            num_e = s[s.find("(")+1:s.find(")")] #number in parentheses
            num_zeroes = len(num_w) - len(num_w.rstrip('0'))
            num_e = '(' + num_e + '0'*num_zeroes + ')'
            value_wl = num_w+num_e
            #value_wl = s.split('(')[0] + num_e
            #print(p, test_val, test_err, value_wl, num_zeroes, num_w)

        else: #small enough to not use sci.not
            one_dig = to_one_dig(test_val, test_err)
            two_dig = round(test_val, test_err, format = 'Drake') 

            if test_err == 0: #if no error use 4 digit wavelength 
                value_wl = all_state.wavelength[p]

            #if two dig has error something like (97), one_dig has error (1). It rounds to 10 but then cuts the 0, which appears wrong
            #needs to be pushed up one digit spot. I.e. (2.45 (97) needs to be 2.5(1))
            elif one_dig[one_dig.find("(")+1:one_dig.find(")")][0] < two_dig[two_dig.find("(")+1:two_dig.find(")")][0]: 
                left_sigfs = len(one_dig.split('(')[0][0:-1]) - 1 #rounding to one less digit instead of slicing for rouding
                #value_wl = round(one_dig.split('(')[0], sigfigs = left_sigfs) + '(' + one_dig[one_dig.find("(")+1:one_dig.find(")")][0] + ')' #parentheses and number in parentheses
                
                #Rounds original number to correct number of sig figs
                value_wl = str(round(test_val, sigfigs = left_sigfs)) + '(' + one_dig[one_dig.find("(")+1:one_dig.find(")")][0] + ')'  
            #otherwise, standard case
            else: 
                if test_err < .0001:
                    value_wl = to_one_dig_small(test_val, float(all_state.Eerr[p]))
                else:
                    value_wl = to_one_dig_small(test_val, test_err)

        
        

    value_tr = round(str(save_copy['transition_rate s-1'][p]), str(save_copy.Terr[p]), format = "Drake")
    if 'E' in value_tr:
        pass
    else:
        numt = "{:E}".format(Decimal(value_tr.split('(')[0])).split('E')
        if float(numt[1]) < 10: #need to add '0' in front of value if exponent power is less than 10
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + '+' + '0' + str(int(numt[1]))
            
        else:
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + numt[1]
    
    value_br = round(str(save_copy['branching ratio'][p]), str(save_copy.Berr[p]), format = "Drake")
    holder.append((value_wl, value_tr, value_br))
holder
save_copy['wavelength'] = [i[0] for i in holder]
save_copy['transition_rate s-1'] = [i[1] for i in holder]
save_copy['branching ratio'] = [i[2] for i in holder]

C:\Users\dmgame\Documents\SafronovaResearch\LifetimesWebsite\modsigfig.py:572: UserWarning: warning: 2 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


In [24]:
af_format = save_copy.copy()

In [27]:
#changes out of scientific notation if the error in parentheses (###) could be 3 or less digits
#transition rates

for j, i in enumerate(save_copy['transition_rate s-1']):
    if 'E' in i:
        #print(i)
        num_in_par = '(' + re.search('\(([^)]+)', i).group(1) + ')' #number in parentheses
        len_sfigs = len(i.split('(')[0]) - 1 #number of digits before expontial, minus the decimal point
        float_num = round(i.replace(num_in_par, ''), sigfigs = len_sfigs) #number in float format
        num_zeroes = len(str(i.split('.')[0])) - (len(num_in_par) - 2) #difference in original number digits - two error format digits
        new_error = re.search('\(([^)]+)', i).group(1) + '0'*num_zeroes #error with correct number of '0's
        if len(new_error) > 4: #keep it in scientific notation if you have to go past 4 digits in error parentheses
            save_copy.loc[j,'transition_rate s-1'] = float_num + '(' + new_error + ')'
            print(new_error, float_num + '(' + new_error + ')')
        else:
            pass

In [ ]:
#replaces scientific notation if the error number (###) is 3 or less digits. keeps if its 4 (####) or more
for j, i in enumerate(Lifetimes.Lifetime):
    if 'E' in i:
        print(i)
        num_in_par = '(' + re.search('\(([^)]+)', i).group(1) + ')' #number in parentheses
        len_sfigs = len(i.split('(')[0]) - 1 #number of digits before expontial, minus the decimal point
        float_num = round(i.replace(num_in_par, ''), sigfigs = len_sfigs) #number in float format
        num_zeroes = len(str(life_linear[j][2]*10**9).split('.')[0]) - (len(num_in_par) - 2) #difference in original number digits - two error format digits
        new_error = re.search('\(([^)]+)', i).group(1) + '0'*num_zeroes #error with correct number of '0's
        if len(new_error) < 4: #keep it in scientific notation if you have to go past 4 digits in error parentheses
            Lifetimes.loc[j,'Lifetime'] = float_num + '(' + new_error + ')'
            print(j, float_num + '(' + new_error + ')')
        else:
            pass

In [28]:
test = formatter(save_copy)
save_copy = test

d:\python3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [29]:
Lifetime_excel = Lifetimes.copy()
Lifetimes2 = Lifetimes.copy()

In [30]:
tl1, tl2, tl3 = format_lifetime(Lifetimes, life_linear)
Lifetimes = tl1
Lifetime_excel = tl3

In [31]:
df2 = pd.read_fwf(checklifesname, dtype = str)

col_names = ['Transition',     'Wavelength',     'Matrix', 'Reference',    
             'Br. ratio',  'Tran. rate',    'wave2',     'UncW', 'UncW %', 'Matrix el.', 
             'uncM', 'uncM %', 'Br2', 'uncB', 'uncB %', 'Tran2',  'UncT',   'UncT %'] 
             
df = pd.read_fwf(checkratesname, header=None, dtype = str)
df.columns = col_names
keeptemp = df['Transition'].str.split(" +",expand = True)


new_cols = ['Initial', 'Decay']
for i in list(df.columns):
    new_cols.append(i)
df['Initial'] = keeptemp[0]
df['Decay'] = keeptemp[1]
df = df[new_cols]
df.drop(0, axis = 0, inplace = True)
df.drop(columns = ['Transition'], axis = 1, inplace = True)
df.reset_index(drop = True, inplace = True)

In [32]:
pd.read_fwf(checkratesname, header=None, dtype = str)[328:330]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
328,10d5/2 5f7/2,2253(2),.44008(72),NaN,0.01495(23),6.127(18)E+04,2253,2,0.09,1.44008,.00072,0.05,0.0149,0.0002,1.53,6.1274E+04,1.8E+02,0.29
329,10d5/2 6f5/2,5183(5),1.3148(50),NaN,.001023(18),4.192(35)E+03,5183,5,0.10,1.3148,0.0050,0.38,0.00102,0.00001,1.72,4.1923E+03,3.5E+01,0.82


In [33]:
#Wavelengths
diff_holdt = []
for i in range(len(save_copy)):
    t_ind = np.where((save_copy.Initial == df.Initial[i]) & (save_copy.Decay == df.Decay[i]))[0][0] #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    if holder[t_ind][0] == df.Wavelength[i]:
        pass
    else:
        diff_holdt.append((i, t_ind, holder[t_ind][0], df.Wavelength[i], all_state.wavelength[t_ind], all_state.Eerr[t_ind]))
diff_holdt

[(263, 279, '28300(800)', '28300(80)', '28295.3949', '80.0644'),
 (298, 357, '488000(1000)', '88(1)E+05', '487567.0405', '1197.1355'),
 (330, 267, '17700(300)', '17700(30)', '17703.7619', '34.4771'),
 (337, 304, '187000(2000)', '187000(200)', '186975.3006', '176.0529'),
 (360, 325, '12800(200)', '12800(20)', '12802.2155', '16.3900')]

In [39]:
#should it be in range len(df)?
#Matrices
diff_holdt = []
for i in range(len(save_copy)):
    t_ind = np.where((save_copy.Initial == df.Initial[i]) & (save_copy.Decay == df.Decay[i]))[0][0] #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    if save_copy['Matrix element (a.u.)'][t_ind].lstrip('0').lstrip('.').lstrip('0') == df.Matrix[i].lstrip('0').lstrip('.').lstrip('0'):
        pass
    else:
        diff_holdt.append((i, t_ind, save_copy['Matrix element (a.u.)'][t_ind], df.Matrix[i], all_state.matrix[t_ind], all_state.mat_unc[t_ind]))
diff_holdt

[(56, 56, '4.277(8)E37', '4.277(8)', 4.277, 0.008),
 (120, 57, '5.898(15)E37', '5.898(15)', 5.898, 0.015),
 (281,
  309,
  '11.5937(58)',
  '1.5937(58)',
  Decimal('11.5937180'),
  Decimal('0.0057969')),
 (303,
  359,
  '15.7365(79)',
  '5.7365(79)',
  Decimal('15.7365020'),
  Decimal('0.0078683')),
 (327,
  273,
  '1.44008(72)',
  '.44008(72)',
  Decimal('1.4400843'),
  Decimal('0.0007200'))]

In [40]:
#only these values have a 1 in the far left position space of the columns. Need to change inferred column width?

In [41]:
#Transition Rates
diff_holdt = []
for i in range(len(save_copy)):
    t_ind = np.where((save_copy.Initial == df.Initial[i]) & (save_copy.Decay == df.Decay[i]))[0][0] #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    #print(holder[t_ind][2].lstrip('0').lstrip('.').lstrip('0'), df['Br. ratio'][i].lstrip('0').lstrip('.').lstrip('0'))
    if holder[t_ind][1] == df['Tran. rate'][i]:
        pass
    else:
        diff_holdt.append((i, t_ind, holder[t_ind][1], df['Tran. rate'][i], all_state['transition_rate s-1'][t_ind], all_state.Terr[t_ind]))
diff_holdt

[]

In [42]:
#branching ratios
diff_holdt = []
for i in range(len(save_copy)):
    t_ind = np.where((save_copy.Initial == df.Initial[i]) & (save_copy.Decay == df.Decay[i]))[0][0] #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    #print(holder[t_ind][2].lstrip('0').lstrip('.').lstrip('0'), df['Br. ratio'][i].lstrip('0').lstrip('.').lstrip('0'))
    if holder[t_ind][2].lstrip('0').lstrip('.').lstrip('0') == df['Br. ratio'][i].lstrip('0').lstrip('.').lstrip('0'):
        pass
    else:
        diff_holdt.append((i, t_ind, holder[t_ind][2], df['Br. ratio'][i], all_state['branching ratio'][t_ind], all_state.Berr[t_ind]))
diff_holdt

[]

In [44]:
#Lifetimes
diff_holdt = []
for i in range(len(df2)):
    t_ind = np.where(Lifetimes.State == df2.State[i]) #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    #print(holder[t_ind][2].lstrip('0').lstrip('.').lstrip('0'), df['Br. ratio'][i].lstrip('0').lstrip('.').lstrip('0'))
    if Lifetimes.Lifetime[t_ind[0][0]] == df2['Lifetime'][i]:
        pass
    else:
        diff_holdt.append((i, t_ind, Lifetimes.Lifetime[t_ind[0][0]], df2['Lifetime'][i]))
diff_holdt

[(7, (array([7], dtype=int64),), '29.45(11)*', '29.45(11)'),
 (15, (array([8], dtype=int64),), '21.02(11)*', '21.02(11)')]

In [45]:
Life_holder = Lifetimes.copy()

In [46]:
#LOOK AT THIS

In [48]:
# save_copy.to_csv('Transition_Formatted.txt', sep='\t', index = False)
# Lifetimes.to_csv('Lifetime_Formatted.txt', sep='\t', index = False)

In [49]:
#excel copy is the version that will be saved when they click "excel"
# excel_copy = all_state.copy()
# for i in range(len(excel_copy)-1, 0, -1):
#     if excel_copy.Initial[i][1] >= 3:
#         excel_copy.drop(i, axis = 0, inplace = True)
# excel_copy['Initial'] = save_copy['Initial']
# excel_copy['Decay'] = save_copy['Decay']
# excel_copy.drop(['Ei', 'Ef', 'Ei_unc', 'Ef_unc', 'mat_werr', 'old_unc', 'n', 'l', 's'], axis = 1, inplace = True)
# excel_copy.rename(columns = {'matrix': 'Matrix Element (a.u.)', 
#                              'mat_unc': 'Matrix Error', 'wavelength': 'Wavelength (nm)', 'Eerr': 'Wavelength Error',
#                              'transition_rate s-1': 'Transition Rate (Hz)', 'Terr': 'Transition Rate Error',
#                             'branching ratio': "Branching Ratio", 'Berr': "Branching Ratio Error"}, inplace = True)
#excel_copy['Wavelength (nm)'] = excel_copy['Wavelength (nm)']*10**7 #nm conversion
#excel_copy

In [50]:
# #no_errors is the version that will be shown when they click "all"
# no_errors = save_copy.copy()
# matrixx = []
# wavell = []
# transrr = []
# branchh = []
# for i in range(len(no_errors)):
#     matrixx.append(no_errors['Matrix element (a.u.)'][i].split('(')[0])
#     wavell.append(no_errors['Wavelength (nm)'][i].split('(')[0])
#     transrr.append(no_errors['Transition Rate (s-1)'][i].split('(')[0] + no_errors['Transition Rate (s-1)'][i].split(')')[1])
#     branchh.append(no_errors['Branching ratio'][i].split('(')[0])
# no_errors['Matrix element (a.u.)'] = matrixx
# no_errors['Wavelength (nm)'] = wavell
# no_errors['Transition Rate (s-1)'] = transrr
# no_errors['Branching ratio'] = branchh
#no_errors

In [51]:
#sanity check that no branching ratio is greater than 1
for i,j in enumerate(no_errors['Branching ratio']):
    if float(j) > 1: 
        print(i, j, no_errors['Initial'][i], no_errors['Decay'][i], 'BRANCHING RATIO > 1!')
        break

In [52]:
# Br_not_sci = [] #to get branching ratios out of scientific notation in html
# Brer_not_sci = []
# for i in range(len(excel_copy)):
#     Br_not_sci.append(str(excel_copy['Branching Ratio'][i]))
#     Brer_not_sci.append(str(excel_copy['Branching Ratio Error'][i]))
# excel_copy['Branching Ratio'] = Br_not_sci
# excel_copy['Branching Ratio Error'] = Brer_not_sci

In [79]:
#t_scifi2 = pd.DataFrame(t_scifi)
# html5 = t_scifi2.to_html(index = False)
# with open('t_scifi2.html', 'w') as fo:
#     fo.write(html5)

NameError: name 't_scifi' is not defined

In [ ]:
html = save_copy.to_html(index = False)
html2 = Lifetimes.to_html(index = False)
html3 = no_errors.to_html(index = False)
html4 = excel_copy.to_html(index = False)
#print(html)

In [ ]:
sub_scripts = ['1/2', '3/2', '5/2', '7/2', '9/2', '11/2']
htmls = [html, html2, html3, html4] #save_copy, Lifetimes, no_error, excel_copy
for i in sub_scripts:
    for j, k in enumerate(htmls):
        htmls[j] = htmls[j].replace('%s' % i, '<sub>%s</sub>' % i)

In [ ]:
with open('save_copy.html', 'w') as fo:
    fo.write(htmls[0])

In [ ]:
with open('Lifetimes.html', 'w') as fo:
    fo.write(htmls[1])

In [ ]:
with open('no_errors.html', 'w') as fo:
    fo.write(htmls[2])

In [ ]:
with open('excel_copy.html', 'w') as fo:
    fo.write(htmls[3])

In [ ]:
#my code goes has 9p1/2 to 7s1/2 before 9p1/2 to 6s1/2
# nsi = []
# lsi = []
# ssi = []
# nsd = []
# lsd = []
# ssd = []
# for i in range(len(all_state)):
#     ns.append(all_state.Initial[])

In [ ]:
#if the error is .0099
#roudning of the number
#branching ratio need to be changed?
#sometimes experimental is uppercase
#make zero zero wavelength to sci. Notation
#codes are in 2020Auto\Codes\IRbis\rates.f
#fortran add ',' in format step (a6, ',')
#add stars to wavelength 
#lifetimes in experimental file are all ns, in future they may be in seconds
#make rates
#copy make file, needs both of experimental files in directory
#./rates
#use rates1 and rate2

In [54]:
def to_one_dig(wavelength, error):
    """
    "takes in a wavelength and error and uses it to produce ####(#) format"
    "could possibly be used to do a different number of (###) digits"
    """
    wavelength = wavelength
    error = error
#     original = round(wavelength, error, format = 'Drake') #number with 2 err_dig and ()   
#     og_err = pattern.findall(original)[0] #just the error if it has one
#     new_err = round(og_err, sigfigs = 1)[0]
        
#     og_num = original.split('(')[0] #just the number  
#     if 'E' in original:
#         err_in_paren = re.search('\(([^)]+)', original).group(1)
#         og_num = str(float(original.replace('(%s)' % err_in_paren, '' ))) #gets it out of scientific notation and back into str
#     new_sfigs = len(og_num.replace('.', '')) - 1 #how many digits new 1 number should have
#     new_num = round(wavelength, sigfigs = new_sfigs) #number moved back one sig fig

#     try:

        
#         num_deci = len(og_num.split('.')[1]) - 1
#     except IndexError:
#         num_deci = new_sfigs
#     deci_format = "%%.%sf" % num_deci
#     new_num = deci_format % new_num
#     #if you want to use scientific notation, keep original as round(wavelength, error, format = 'Drake')

#     final_value = new_num + '(' + new_err + ')'
    ###########
    numb = round(wavelength, error).split('±')[0]
    erb = round(wavelength, error, format = 'Drake')
    erb = re.search('\(([^)]+)', erb).group(1)
    dig1 = round(erb, sigfigs = 1)[0]
    final_value = numb + '(' + dig1 + ')'
    ###############
    if float(error) == 0:
        final_value = wavelength
    return final_value


'28300(8)'

In [ ]:
Everything is working, except for the thing being read in wrong

In [76]:
lst1 = [(5, 2, '4968(1)', '4968.5(1)', '4968.4652', '0.0992'),
 (49, 45, '4399(1)', '4398.7(1)', '4398.7464', '0.0974'),
 (116, 186, '4368(1)', '4367.9(1)', '4367.8556', '0.0963'),
 (205, 197, '4367(1)', '4367.4(1)', '4367.4359', '0.0963'),
 (263, 279, '28300(800)', '28300(80)', '28295.3949', '80.0644'),
 (270, 317, '4362(1)', '4362.4(1)', '4362.3813', '0.0958'),
 (298, 357, '488000(1000)', '88(1)E+05', '487567.0405', '1197.1355'),
 (330, 267, '17700(300)', '17700(30)', '17703.7619', '34.4771'),
 (337, 304, '187000(2000)', '187000(200)', '186975.3006', '176.0529'),
 (360, 325, '12800(200)', '12800(20)', '12802.2155', '16.3900')]
# for p in range(279,280):
#     ini = all_state.Initial[p]
#     end = all_state.Decay[p]
#     test_val = all_state.precise_wave[p]
#     test_err = all_state.precise_Eerr[p]
# for i in lst1:
#     test_val = all_state.precise_wave[i[1]]
#     test_err = all_state.precise_Eerr[i[1]]
holder = []
for p in range(len(all_state)):
#for p in range(167, 168):
    test_val = all_state.precise_wave[p]
    test_err = all_state.precise_Eerr[p]
    
    if float(test_err) == 0:
        value_wl = '%.4f' % test_val
    else:
        #condition 1 is for wavleenghts and Errors > 9.5
        if float(all_state.wavelength[p]) and float(all_state.Eerr[p]) > 9.5: #need to avoid sci. notat
            #len_figs = the difference between the digit placement of the number and its error. I.e. ####.## - ##.## will be 2
            len_figs = len(str(test_val).split('.')[0]) - len(str(test_err).split('.')[0])

            if float(test_err) < 10: #9.5-10 will round up to 1 and lose a zero digit
                num_w = str(round(str(test_val), sigfigs = len_figs)).split('.')[0] #dont need to add one if it isnt double digits
                added_digit = 1
            else:
                num_w = str(round(str(test_val), len_figs + 1)).split('.')[0]
                added_digit = 0
    
            #number of zeroes is length of digits to the left of the decimal in the error, subtracted by 1
            if float(test_err) > 1: 
                numb = round(test_val, test_err).split('±')[0]
                erb = round(test_val, test_err, format = 'Drake')
                erb = re.search('\(([^)]+)', erb).group(1)
                num_zeroes = len(str(test_err).split('.')[0]) - 1 + added_digit
                num_e = '(' + round(erb, sigfigs = 1)[0] + '0'*num_zeroes + ')'
                value_wl = num_w + num_e
            else:
                value_wl = to_one_dig(test_val, test_err) 
        #condition 2 is for Errors smaller than 9.5
        else: #small enough to not use sci.not
            one_dig = to_one_dig(test_val, test_err)
            two_dig = round(test_val, test_err, format = 'Drake') 

            if test_err == 0: #if no error use 4 digit wavelength 
                value_wl = all_state.wavelength[p]

            #if two dig has error something like (97), one_dig has error (1). It rounds to 10 but then cuts the 0, which appears wrong
            #needs to be pushed up one digit spot. I.e. (2.45 (97) needs to be 2.5(1))
            elif one_dig[one_dig.find("(")+1:one_dig.find(")")][0] < two_dig[two_dig.find("(")+1:two_dig.find(")")][0]: 
                
#                 left_sigfs = len(one_dig.split('(')[0][0:-1]) - 1 #rounding to one less digit instead of slicing for rounding
#                 #Rounds original number to correct number of sig figs
#                 value_wl = str(round(test_val, sigfigs = left_sigfs)) + '(' + one_dig[one_dig.find("(")+1:one_dig.find(")")][0] + ')'  
                value_wl = one_dig
    
            #otherwise, standard case
            else:
                #use the original error if the error is less than the .0001 cutoff, since otherwise error will not show up
                if test_err < .00005:
                    value_wl = all_state.wavelength[p]
                    #code breaks if you pass in 0 error
                #smaller than .0001, needs to round to 4 digits
                elif test_err < .0001:
                    value_wl = to_one_dig(test_val, '%.4f' % test_err)
                #any other normal case error
                else:
                    value_wl = to_one_dig_small(test_val, test_err)
                    
    value_tr = round(str(all_state['transition_rate s-1'][p]), str(all_state.Terr[p]), format = "Drake")
    if 'E' in value_tr:
        pass
    else:
        numt = "{:E}".format(Decimal(value_tr.split('(')[0])).split('E')
        if float(numt[1]) < 10: #need to add '0' in front of value if exponent power is less than 10
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + '+' + '0' + str(int(numt[1]))
            
        else:
            value_tr = numt[0] + '(' + value_tr.split('(')[1] + 'E' + numt[1]
    
    value_br = round(str(all_state['branching ratio'][p]), str(all_state.Berr[p]), format = "Drake")
    holder.append((value_wl, value_tr, value_br))

In [77]:
diff_holdt = []
for i in range(len(save_copy)):
    t_ind = np.where((save_copy.Initial == df.Initial[i]) & (save_copy.Decay == df.Decay[i]))[0][0] #same transition
    #print(save_copy['Wavelength (nm)'][t_ind], df['Wavelength'][i])
    if holder[t_ind][0] == df.Wavelength[i]:
        pass
    else:
        diff_holdt.append((i, t_ind, holder[t_ind][0], df.Wavelength[i], all_state.wavelength[t_ind], all_state.Eerr[t_ind]))
diff_holdt

[(298, 357, '488000(1000)', '88(1)E+05', '487567.0405', '1197.1355')]